In [107]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.datasets import mnist
from keras.models import Sequential,model_from_json
from keras.layers import Dense
from tensorflow.keras.optimizers import RMSprop
from keras.utils import np_utils
import pylab as plt
from keras.regularizers import l1,l2
import cv2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.vgg16 import preprocess_input
from keras.layers import Dense, Dropout
from tensorflow.keras import layers, models
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input


In [106]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm

In [71]:
#loading and preprocessing MNIST
(x_train,y_train), (x_test, y_test)= mnist.load_data()

In [72]:
x_train = np.repeat(x_train[..., np.newaxis],3,-1)
x_test = np.repeat(x_test[..., np.newaxis],3,-1)
x_train = np.array([cv2.resize(img,(48,48)) for img in x_train])
x_test = np.array([cv2.resize(img,(48,48)) for img in x_test])

In [73]:
x_train = preprocess_input(x_train)
x_test = preprocess_input(x_test)

In [74]:
#normalizing the Data
x_train= x_train/255
x_test= x_test/255

In [75]:
#one hot encoding the data
y_train_hcoded=to_categorical(y_train,10)
y_testhcoded=to_categorical(y_test,10)

In [76]:
## Loading VGG16 model
vgg16= VGG16(weights="imagenet", include_top=False,classes=10, input_shape=x_train[0].shape)

In [87]:
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 24, 24, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 24, 24, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 12, 12, 128)       0     

### Random Forest

In [77]:
layername= 'block5_conv2'
layer = vgg16.get_layer(layername)

In [78]:
from keras.models import Model
model = Model(inputs=vgg16.input, outputs=layer.output)

In [79]:
features_train = model.predict(x_train)
features_test = model.predict(x_test)

313/313 [==============================] - 3s 10ms/step


In [80]:
#initializing the Random Forest Model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

In [81]:
# Flatten the extracted features
features_train_flat = features_train.reshape(features_train.shape[0], -1)
features_test_flat = features_test.reshape(features_test.shape[0], -1)

In [118]:
features_test_flat.shape


(10000, 512)

In [82]:
rf_model.fit(features_train_flat, y_train)

RandomForestClassifier(random_state=42)

In [84]:
rf_modelpredict= rf_model.predict(features_test_flat)

In [92]:
rfaccuracy = rf_model.score(features_test_flat, y_test)
print("Random Forest accuracy =", rfaccuracy)


Random Forest accuracy = 0.9746


### Logistic Regression



In [88]:
layername = 'block5_pool'
layer2 = vgg16.get_layer(layername)

In [90]:
from keras.models import Model
model = Model(inputs=vgg16.input, outputs=layer2.output)

In [91]:
features_train = model.predict(x_train)
features_test = model.predict(x_test)

313/313 [==============================] - 3s 11ms/step


In [100]:
#initializing the Logistic Regression Model
lr_model = LogisticRegression(C=0.01, solver='lbfgs', max_iter=1000)

In [101]:
# Flatten the extracted features
features_train_flat = features_train.reshape(features_train.shape[0], -1)
features_test_flat = features_test.reshape(features_test.shape[0], -1)

In [102]:
lr_model.fit(features_train_flat, y_train)

LogisticRegression(C=0.01, max_iter=1000)

In [103]:
lr_modelpredict= lr_model.predict(features_test_flat)

In [104]:
lraccuracy = lr_model.score(features_test_flat, y_test)
print("Logistic Regression accuracy =", lraccuracy)

Logistic Regression accuracy = 0.95


### SVM

In [108]:
layername= 'block5_pool'
layer = vgg16.get_layer(layername)

In [109]:
# Define a new model that outputs the layer output
model = Model(inputs=vgg16.input, outputs=layer.output)

In [110]:
# Extract features from the intermediate layer for both training and testing data
features_train = model.predict(x_train)
features_test = model.predict(x_test)

313/313 [==============================] - 3s 11ms/step


In [111]:
# Flatten the extracted features
features_train_flat = features_train.reshape(features_train.shape[0], -1)
features_test_flat = features_test.reshape(features_test.shape[0], -1)

In [112]:
# Initialize the SVM model
svm_model = svm.SVC(kernel='linear', C=1, gamma='auto')

In [113]:
# Train the SVM model on the flattened features and corresponding labels
svm_model.fit(features_train_flat, y_train)

SVC(C=1, gamma='auto', kernel='linear')

In [114]:
svm_modelpredict= svm_model.predict(features_test_flat)

In [115]:
# Evaluate the SVM model on the test set
accuracy = svm_model.score(features_test_flat, y_test)
print("SVM accuracy =", accuracy)

SVM accuracy = 0.9812
